In [1]:
import geopandas as gp
import osmnx as ox
import numpy as np
import time
import csv
import os

import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.geometry import Polygon
from math import radians, cos, sin, asin, sqrt

edge_dataframe = gp.GeoDataFrame.from_file('data/porto/edges.shp')
print('Total edges in the graph:' , len(edge_dataframe))
edge_dataframe.head()

nodes_dataframe = gp.GeoDataFrame.from_file('data/porto/nodes.shp')
#print('Total nodes in the graph: ', len(nodes_dataframe))
#nodes_dataframe.head()

Total edges in the graph: 189876


In [2]:


class Edge(object):
    def __init__(self, idx, u, v, geometry):
        self.idx = idx
        self.u = u
        self.v = v
        self.geometry = geometry

def get_edge_from_idx(index):
    edge_data = edge_dataframe.iloc[index]
    u_pos = np.flatnonzero(nodes_dataframe['osmid']==edge_data.u)
    node_u_data = nodes_dataframe.iloc[u_pos]
    v_pos = np.flatnonzero(nodes_dataframe['osmid']==edge_data.v)
    node_v_data = nodes_dataframe.iloc[v_pos]
    
    u = (node_u_data.x.iloc[0], node_u_data.y.iloc[0])
    v = (node_v_data.x.iloc[0], node_v_data.y.iloc[0])
    geo = edge_data.geometry
    
    
    edge = Edge(index, u, v, geo)
    
    return edge

def dict_add_n(dict_to_add, key, n):
    if key in dict_to_add.keys():
        dict_to_add[key] += n
    else:
        dict_to_add[key] = n

def haversine_dist(coord1, coord2):
    lon1, lat1 = coord1[0], coord1[1]
    lon2, lat2 = coord2[0], coord2[1]
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def get_edge_timeslice():
    csv_data = open('data/matched_routines.csv', 'r')
    dict_reader = csv.DictReader(csv_data)
    
    edge_timeslice_list = []
    edge_timeslice_dict = {}
    
    processing_count = 1
    with tqdm(total=662) as _tqdm:
        for item in dict_reader:
            _tqdm.set_description('processing: {}/{} trajectory'.format(processing_count, 662))
            trajectory_edge_timeslice = {}
            if item['match_success'] == 'FALSE':
                continue
            raw_gps_trajectories = eval(item['raw_gps_trajectories'])
            edge_seq = eval(item['fmm-mapped edge indice sequence(map edge id)(equivalent to cpath from FMM)'])

            g_pointer = 0
            e_pointer = 0


            while g_pointer < len(raw_gps_trajectories):
                edge = get_edge_from_idx(edge_seq[e_pointer])
                gps_coord = raw_gps_trajectories[g_pointer]

                if e_pointer == len(edge_seq) - 1:
                    dict_add_n(trajectory_edge_timeslice, edge.idx, 1)
                    g_pointer += 1
                    continue

                if haversine_dist(gps_coord, edge.u) > haversine_dist(edge.u, edge.v):
                    e_pointer += 1
                    edge = get_edge_from_idx(edge_seq[e_pointer])

                dict_add_n(trajectory_edge_timeslice, edge.idx, 1)
                g_pointer += 1
                
            _tqdm.update(1)
            processing_count += 1
            edge_timeslice_list.append(trajectory_edge_timeslice)
        
    edge_timeslice_dict = compute_edge_timeslice(edge_timeslice_list)
    return edge_timeslice_dict

def compute_edge_timeslice(edge_timeslice_list):
    edge_timeslice_dict = {}
    for i in range(0, len(edge_dataframe)):
        count = 0
        for item in edge_timeslice_list:
            if i in item.keys():
                dict_add_n(edge_timeslice_dict, i, item[i])
                count += 1
        if i in edge_timeslice_dict.keys():
            edge_timeslice_dict[i] = edge_timeslice_dict[i] / count
    
    return edge_timeslice_dict  

In [3]:
edge_timeslice = get_edge_timeslice()

processing: 1/662 trajectory:   0%|          | 0/662 [00:00<?, ?it/s]


KeyError: 'match_success'